<a href="https://colab.research.google.com/github/AlekZivkovic/pp_CUDA_matrix_multiplication/blob/main/Treci_Projekat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyCuda

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 75 kB 5.1 MB/s 
     |████████████████████████████████| 66 kB 6.3 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626633 sha256=8a635e9bd735c07076b904f727980b11c74812127fe727391f328dc61dee8946
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.9-py2.py3-none-any.whl size=62370 sha256=f18cf3f5ece2a73af8f3c4fa10f20e38b0701a826ac32e9ba11a6ddb1b91b74a
  Stored in directory: /root/.cache/pip/wheels/41/b9/6e/94bb014f6484b15ec77e7877f3a227609481ffd98db364504d
Successfully built pycuda pytools


In [ ]:
!nvidia-smi

Thu Jan 13 15:58:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Cuda Funkcije

In [ ]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule("""
#include <stdio.h>

    //Trazi trenutni element iz a i b i vraca njihov proizvod
    //Koristim za svaki element reda iz matrice A i kolone iz matrice B
    __device__ float find_in_a_and_b(int transa, int transb, int n_a, int n_b, float* A, float* B, int i){

        int index_a, index_b, x_a, y_a, x_b, y_b;
        
        //Trazim trenutni element iz matrice a
        //Ako nije transponovana idem po redu, a ako jeste po koloni y (y jer je to red trenutnog polja matrice c, treba da idem po tom redu, odnosno koloni, matrice a)
        
        //Ako matrica a nije transponovana zakucam red i pomeram se po njemu u odnosu na trenutno mesto koje gledam(i), a ako jeste zakucam kolonu i pomeram se po njoj
        //Na kraju uzmem index u matrici a (trenutni red * velicina tog red(broj kolona) + trenuna kolona)
        //Zakucavam red ili kolonu po tome u kojem se redu nalazi trenutna nit
        if(!transa){
            x_a = i;
            y_a = threadIdx.y + blockIdx.y*blockDim.y;
        }
        else{
            y_a = i;
            x_a = threadIdx.y + blockIdx.y*blockDim.y;
        }
        index_a = y_a*n_a + x_a;

        //Ista logika kao za matricu a, samo sto zakucavam kolonu ili red u odnosu na kolonu u kojoj se nalazi trenutna nit
        if(!transb){
            y_b = i;
            x_b = threadIdx.x + blockIdx.x*blockDim.x;
        }
        else{
            x_b = i;
            y_b = threadIdx.x + blockIdx.x*blockDim.x;
        }
        index_b = y_b*n_b + x_b;

        //Vracam proizvod elementa iz matrice a i elementa iz matrice b
        return A[index_a] * B[index_b];
    }

    //transa: [1] - transponovana je, [0] - nije transponovana
    __global__ void gemm_no_shared(int transa, int transb, int m_a, int n_a, int k, float alpha, float* A, float* B, float* C){
        
        __shared__ int m_c, n_c, m_b, n_b;
        
        //Uzimam velicine b i c matrice, cisto da bi mi bilo lakse, ubacujem to u shared memoriju da ne bi svaki thread morao da racuna za sebe, prvi thread u bloku izracuna dimenzije
        //Nakon sto prvi thread izracuna dimenzije, syncujem niti da bi svaka nit imala pristup dimenzijama
        if(threadIdx.x == 0 && threadIdx.y == 0){
            //Gledam na sta se odnosi k
            if(!transb)
                n_b = k;
            else
                m_b = k;
            
            //Gledam drugu dimenziju matrice b, odnosno unutrasnju vrednost koja se poklapa sa nekom od dimenzija matrice a
            if(!transa && !transb)
                m_b = n_a;
            else if(transa && !transb)
                m_b = m_a;
            else if(!transa && transb)
                n_b = n_a;
            else
                n_b = m_a;

            //Po dimenzijama matrica a i b racunam dimenzije matrice c
            m_c = (transa == 1)? n_a: m_a;
            n_c = (transb == 1)? m_b: n_b;
        }
        __syncthreads();

        //Uzimam x i y koordinatu trenutnog threada, odnosno trenutnog polja matrice c, ako ispadam iz matrice radim return
        int x = blockDim.x*blockIdx.x + threadIdx.x;
        int y = blockDim.y*blockIdx.y + threadIdx.y;
        
        if(x >= n_c || y >= m_c)
            return;
        
        int index_c = y*n_c + x;
        
        //Uzimam unutrasnju vrednost kao limit, pokazuje mi koliko elemenata treba da gledam za trenutno polje, odnosno koliko elemenata iz matrica a i b treba da pomnozim
        int limit = (transa == 0)? n_a: m_a;
        float sum = 0;
        //Prolazim kroz red matrice a i kolonu matrice b i u svakoj iteraciji dodajem na sumu proizvod trenutnih elemenata matrica
        for(int i = 0; i<limit; i++){
            sum += find_in_a_and_b(transa, transb, n_a, n_b, A, B, i);
        }
        C[index_c] = sum*alpha;
    }

  //matTip: [0] - matrica a, [1] - matrica b
  //block_offset - blok koji se trenutno gleda, odnosno blok koji se trenutno ubacuje u shared niz
  //i_cord - red trenutnog threada, j_cord - kolona trenutnog threada
  __device__ float get_valueV2(int matTip,int transflag, int colum_len,int row_len,int j_cord,int i_cord,int block_offset,float *matrix){

       int idx;

       //Uslovi u svakom ifu proveravaju da li vrednosti koja se gleda ispada iz matrice (da li je poslednji blok, i ako jeste da li bi trenutni thread iz c matrice postojao u tom bloku ili ne)

       //Ako se gleda matrica a i nije transponovana, uzimamo vrednost iz trenutnog bloka, koji se pomera po redovima, koja se nalazi na poziciji trenutnog threada
       if(matTip == 0 && transflag==0 && blockDim.x*block_offset + threadIdx.x < colum_len){
          idx=i_cord * colum_len + (block_offset * blockDim.x + threadIdx.x);
            return matrix[idx];
       }

       //Ako se gleda matrica a i transponovana je, uzimamo vrednost iz trenutnog bloka, koji se pomera po kolonama, koja se nalazi na poziciji trenutnog threada
       if(matTip == 0 && transflag==1 && blockDim.x*block_offset + threadIdx.x < row_len){
          idx= (block_offset * blockDim.y +threadIdx.x) * colum_len + i_cord;
          return matrix[idx];
       }

      if(matTip==1 && transflag==0 && blockDim.x * block_offset + threadIdx.y < row_len){
            
            idx = ( block_offset * blockDim.y + threadIdx.y) * colum_len + j_cord; 
            return matrix[idx];           
      }
      
      //Ukoliko imamo vise blokova moramo voditi racuna o tome da li nasa vrednost ispada iz matrice b,
      //odnosno da li ispada iz reda koji se gleda
      if(matTip == 1 && transflag==1 && block_offset * blockDim.x + threadIdx.y < colum_len){
          idx=j_cord * colum_len + (block_offset * blockDim.y + threadIdx.y);
          return matrix[idx];
      }

       return 0;
    }

    //Prvi deo funkcionise na isti nacin kao gemm_no_shared
    __global__ void gemm_shared(int transa, int transb, int m_a, int n_a, int k, float alpha, float* A, float* B, float* C){
        __shared__ int m_c, n_c, m_b, n_b;
        //postavljamo odgovarajuce vrednosti granica nasih matrica
        if(threadIdx.x == 0 && threadIdx.y == 0){
            if(!transb)
                n_b = k;
            else
                m_b = k;
            
            if(!transa && !transb)
                m_b = n_a;
            else if(transa && !transb)
                m_b = m_a;
            else if(!transa && transb)
                n_b = n_a;
            else
                n_b = m_a;

            m_c = (transa == 1)? n_a: m_a;
            n_c = (transb == 1)? m_b: n_b;
        }
        __syncthreads();

        //pronalazimo x_cord & y_cord za nit
        int x = blockDim.x*blockIdx.x + threadIdx.x;
        int y = blockDim.y*blockIdx.y + threadIdx.y;
       
        //pronalazimo index za koji element nit radi
        //n_c je br kolona elem sama sirina matrice
        int index_c = y*n_c + x;

        float sum = 0;

        
        int limit = (transa == 0)? n_a: m_a;
        //Nizovi koji predtavljaju izvucene vrednosti iz matrice a i iz matrice b 
        __shared__ float a_spart[1024];
        __shared__ float b_spart[1024];

       // Pronalazimo ukupni broj blokova koji treba da se prodju po unutrasnjoj dimenziji matrica, posto je celobrojno deljenje moramo da zaokruzimo
        int block_number= limit / blockDim.x; 
        //zaokruzujemo na vecu decimalu
        block_number= limit % blockDim.x ? block_number + 1 : block_number;
        
        //Prolazicemo kroz sve blokove koji ce nam trebati za trenutni blok koj se izvrsava 
        //(ako smo u bloku 0,0 prolazicemo kroz sve blokove iz matrice a koje predstavljaju redove 0-32 i sve blokove koji predstavljaju kolone 0-32 u matrici b (razlikovace se u odnosu na to da li su transponovane))
        //(ako smo u bloku 0, 1 sve redove koji su 0-32 i sve kolone koje su 33-64)
        //Svaki thread ce da ubaci u shared niz element iz bloka koji se nalazi na istom mestu unutar tog bloka kao trenutni thread
        for(int i=0; i < block_number ; i++){
          // posto koristimo 1D niz svaka nit ce na svoje polje ubaciti odg vrednost iz glvne matrice
          a_spart[threadIdx.y*blockDim.x + threadIdx.x]=0;
          b_spart[threadIdx.y*blockDim.x + threadIdx.x]=0;
        
          //Posto get_value svakako gleda da li je matrica transponovana ili nije, indeksi se nece razlikovati (bilo da se u matrici a gledaju redovi ili kolone za mnozenje, u nizu se gleda kao da je red)
          a_spart[threadIdx.y*blockDim.y + threadIdx.x] = get_valueV2(0,transa, n_a,m_a,x,y,i,A);
        
          b_spart[threadIdx.y*blockDim.x + threadIdx.x]= get_valueV2(1,transb, n_b,m_b,x,y,i,B);

          //Threadovi cekaju da svaki ubaci vrednosti u a i b shared niz, sto ce znaciti da trenutno u a i b nizu imamo neki blok koji predstavlja deo redova i kolona koje treba da se pomnoze
          __syncthreads(); 
          int idx_a,idx_b;
          
          //Posto su blokovi 32x32, svaki thread prolazi kroz 32 elementa koja predstavljaju deo elemenata reda matrice a i kolone matrice b koje trenutni thread treba da gleda, mnoze vrednosti koje treba
          //i ubacuju sabiraju ih sa do tada izracunatim vrednostima
          //Ako u trenutnoim blokovima ima manje vrednosti od dimenzije bloka (trenutni blok sadrzi samo 20 elemenata u jednom redu), shared niz za ostale redove ima 0, pa se zbir nece menjati 
          for(int j = 0; j < blockDim.y; j++){

            idx_a= threadIdx.y * blockDim.x + j;
            idx_b=j* blockDim.x + threadIdx.x ;
            
            sum  +=a_spart[idx_a] * b_spart[idx_b] ;
          
          }
          __syncthreads();


        }
        
        //Ako nismo ispali iz matrice c, suma se ubacuje na dobro mesto
        if(x < n_c && y < m_c)
          C[index_c] = sum*alpha;

    }

""")

# Main deo koda

In [ ]:
import numpy as np
import math

def main_func():
    #m_a = int(input('Unesite broj redova matrice A: '))
    #n_a = int(input('Unesite broj kolona matrice A: '))

    #m_b = int(input("Unesite broj redova matrice B: "))
    #n_b = int(input("Unesite broj kolona matrice B: "))

    #trans_a = input('Unesite transA: ').lower()
    #trans_b = input('Unesite transB: ').lower()
    #alpha = int(input('Unesite alpha: '))
    
    #[m] - broj redova, [n] - broj kolona
    #Matrica ce biti mxn
    m_a=300
    n_a=500
    m_b=300
    n_b=500
    trans_a = 'n'
    trans_b = 't'
    alpha = 3

    #Ako se unutrasnje dimenzije ne poklapaju izbacuje se greska
    if (trans_a == 'n' and trans_b == 'n' and n_a != m_b) or (trans_a == 'n' and trans_b == 't' and n_a != n_b) or (trans_a == 't' and trans_b == 'n' and m_a != m_b) or (trans_a == 't' and trans_b == 't' and m_a != n_b):
        print("Nisu dobre dimenzije")
        return;

    #k - vrednost koja se razlikuje, u odnosu na to da li je b transponovano ili ne, uzima se krajnja desna dimenzija jer se unutrasnja dimenzija b poklapa sa unutrasnjom dimenzijom a
    k = n_b if trans_b == 'n' else m_b
    #Uzimam dimenzije matrice c u odnosu na to sta je transponovano, uzimam spoljasnje vrednosti
    m_c = m_a if trans_a == 'n' else n_a
    n_c = n_b if trans_b == 'n' else m_b
    mat_c = np.zeros((m_c, n_c), dtype = np.float32)

    mat_a = np.random.randn(m_a,n_a).astype(dtype=np.float32)
    mat_a = np.round(mat_a, 1)
    mat_b = np.random.randn(m_b,n_b).astype(dtype=np.float32)
    mat_b = np.round(mat_b, 1)
    print(np.matrix(mat_a))
    print(np.matrix(mat_b))

    func = mod.get_function("gemm_shared")
    
    a_gpu = cuda.mem_alloc(mat_a.nbytes)
    b_gpu = cuda.mem_alloc(mat_b.nbytes)
    c_gpu = cuda.mem_alloc(mat_c.nbytes)

    cuda.memcpy_htod(a_gpu, mat_a)
    cuda.memcpy_htod(b_gpu, mat_b)
    #Posto je teze da se posalje karakter funkciji, samo pravim 1 ili 0 kao true ili false, [1] - transponovano, [0] - nije trensponovano
    trans_a_int = 1 if trans_a == 't' else 0
    trans_b_int = 1 if trans_b == 't' else 0
  

    func(np.int32(trans_a_int), np.int32(trans_b_int), np.int32(m_a), np.int32(n_a), np.int32(k), np.float32(alpha), a_gpu, b_gpu, c_gpu, 
         block = (32,32, 1), grid = (math.ceil(mat_c.shape[1]/32), math.ceil(mat_c.shape[0]/32), 1))

    #Ako je neka od matrica transponovana, transponujem je da bih mogao da uradim matmul kako treba
    if trans_a == 't':
        mat_a = np.transpose(mat_a)
    if trans_b == 't':
        mat_b = np.transpose(mat_b)
    #print(np.matrix(mat_a))
    #print(np.matrix(mat_b))
    mul_mat = np.matmul(mat_a, mat_b)
    mul_mat = mul_mat*alpha
    mul_mat = np.round(mul_mat, 2)
    print()
    print(np.matrix(mul_mat))
    print()

    cuda.memcpy_dtoh(mat_c, c_gpu)
    mat_c = np.round(mat_c, 2)
    print(np.matrix(mat_c))
    #Zna da izbaci false jer se desi da omasi u racunanju za neka random polja (za matrice 30x30 se desilo najvise 2 false)
    print((mat_c == mul_mat).all())
    #for i in range (m_c):
     #   for j in range (n_c):
      #      if(mat_c[i][j] != mul_mat[i][j]):
       #         print(i, j)

main_func()

[[ 0.1  0.7 -1.8 ... -0.7 -0.5 -1.2]
 [ 0.2 -0.2 -0.3 ...  0.9 -1.9 -0.8]
 [ 0.4 -0.5 -1.7 ...  1.7  2.1 -1.4]
 ...
 [ 1.  -0.8  0.2 ... -0.  -2.   0.8]
 [-2.1  1.1 -0.1 ...  1.5 -0.9 -0.7]
 [ 0.9 -0.8  0.7 ... -1.5  0.6 -1.9]]
[[ 0.6  0.5  1.  ...  1.4 -0.5  0.9]
 [ 0.2  0.7 -1.  ...  1.8  1.4 -0.4]
 [-0.3 -0.  -0.7 ... -0.4 -1.9 -0.6]
 ...
 [-0.4  0.6  1.8 ...  0.6 -0.5 -0.2]
 [-0.5  1.1  1.5 ... -1.6 -0.4  0.5]
 [-1.8 -1.1  0.2 ... -0.3  0.6 -0.9]]

[[ -37.23    7.65  -70.35 ...   -8.22  -57.81  -58.47]
 [ -54.39  -36.39   67.41 ...   34.02  -40.8    49.44]
 [ 172.41   10.89  -56.58 ...  -96.66   10.35 -163.83]
 ...
 [  57.24  -19.68  112.17 ...  -16.5   -68.43   47.1 ]
 [-108.75    2.31  -37.53 ...  -29.97   65.16   41.7 ]
 [   9.     21.21   43.56 ...   -8.97  -29.73  -95.82]]

[[ -37.23    7.65  -70.35 ...   -8.22  -57.81  -58.47]
 [ -54.39  -36.39   67.41 ...   34.02  -40.8    49.44]
 [ 172.41   10.89  -56.58 ...  -96.66   10.35 -163.83]
 ...
 [  57.24  -19.68  112.17 ...  -16.5